In [3]:
#Hack to fix missing PROJ4 env var
import os
import conda
import random

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

In [4]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

FileNotFoundError: [Errno 2] No such file or directory: '/anaconda3/share/proj/epsg'

In [ ]:
# function to read lat/lon for companies - should have imported the .py file but...
def read_tech_companies():
    ''' Data frame of 10 tech companies
    return pd.DataFrame(tech_company_name, lattitude, longitude)
    '''
    companies = pd.read_csv("tech_companies.csv")
    companies.drop(['ADDRESS'], axis=1, inplace=True)
    
    return(companies)

In [ ]:
# function to read lat/lon for stations - should have imported the .py file but...

def read_mta_stations():
    ''' Read the MTA master list into a pd.DataFrame (Put the CSV file on the repository!!)
    data source = https://data.cityofnewyork.us/Transportation/Subway-Stations/arq3-7z49 (export data as csv)
    '''
    df = pd.read_csv("mta_stations.csv")
    df.rename(columns = {"OBJECTID": "key", "NAME":"name", "the_geom":"location"}, inplace = True)
    
    df["lon"] = df.location.str.split().str.get(1).str.strip('(')
    df["lat"] = df.location.str.split().str.get(2).str.strip(')')
    df = df[["key","name","LINE","lon","lat"]]
    df = df.sort_values(["name","LINE"], ascending = [True, True])
    
    return df
    
    '''
    return pd.DataFrame (columns = [station (unique), lattitude, longitude])
    '''

In [ ]:
# set the data up for companies
companies = read_tech_companies()

comp_lat = []
comp_lon = []
comp_labels = []
for index, row in companies.iterrows():
    comp_lat.append(row['LAT'])
    comp_lon.append(row['LON'])
    comp_labels.append(row['COMPANY'])

In [ ]:
# set the data up for the stations
stations = read_mta_stations()

sta_lat = []
sta_lon = []
sta_vol = []

for index, row in stations.iterrows():
    sta_lat.append(float(row['lat']))
    sta_lon.append(float(row['lon']))
    sta_vol.append(1.)
    
sta_vol[200]=2

In [ ]:
'''
Plots an outline of Manhattan with red dots on the stations and blue dots on the companies

pass the following variables

comp_lat      = list of lattitudes for companies
comp_lon      = list of lattitudes for companies
comp_labels   = list of labels for companies (not currently working)
sta_lat       = list of lattitudes for stations
sta_lon       = list of longitutes for stations

'''

def NYC_map(comp_lat,comp_lon,comp_labels,sta_lat,sta_lon,sta_vol):
    map = Basemap(projection='merc', lat_0 = 40, lon_0 = -74,
        resolution = 'h', area_thresh = 0.02,
        llcrnrlon=-74.04, llcrnrlat=40.68,
        urcrnrlon=-73.93, urcrnrlat=40.79)
    
# Make this plot larger.
    plt.figure(figsize=(16,12))

    map.drawcoastlines()
    map.drawcountries()
    map.fillcontinents(color = 'yellowgreen')
    map.drawmapboundary()

#plot the station points in red
#    x_sta,y_sta = map(sta_lon, sta_lat)
#    map.plot(x_sta, y_sta, 'bo',color='r', markersize=7,label='Stations')

#plot the station points in red with circle varying by station volume
    x_sta,y_sta = map(sta_lon, sta_lat)
    max_dot_size = 15
    dot_factor = max_dot_size/max(sta_vol)
    
    for i in range(len(x_sta)):
        xpt = x_sta[i]
        ypt = y_sta[i]
        mark = int(sta_vol[i]*dot_factor)
        map.plot(xpt, ypt, 'bo',color='r', markersize=mark)

    
#plot the company points in blue
    x_comp,y_comp = map(comp_lon, comp_lat)
    map.plot(x_comp, y_comp, 'bo',color='b', markersize=7, label='Companies')

#create a legend and title
    plt.legend(loc='lower right')
    title_string = "Companies & Stations in NYC"
    plt.title(title_string)

# plot company labels
#    for label, xpt, ypt in zip(comp_labels, x_comp, y_comp):
#        plt.text(xpt, ypt, label)

#    plt.text(comp_labels[0],x_comp[0],y_comp[0])
    
    plt.show()
    return

In [ ]:
# plot all of the companies and all of the labels
NYC_map(comp_lat,comp_lon,comp_labels,sta_lat,sta_lon,sta_vol)


In [ ]:
print(comp_lat[0],comp_lon[0],comp_labels[0],sta_lat[10],sta_lon[10],sta_vol[10])

In [ ]:
# plot just the 0th company and the 10th station

NYC_map(comp_lat[0],comp_lon[0],comp_labels[0],sta_lat[10],sta_lon[10],sta_vol[10])


In [ ]:
# read the station_companies.csv file for values to plot
stations1 = pd.read_csv("station_companies.csv")
#    df.rename(columns = {"OBJECTID": "key", "NAME":"name", "the_geom":"location"}, inplace = True)
    
#    df["lon"] = df.location.str.split().str.get(1).str.strip('(')
#    df["lat"] = df.location.str.split().str.get(2).str.strip(')')
#    df = df[["key","name","LINE","lon","lat"]]
#    df = df.sort_values(["name","LINE"], ascending = [True, True])
stations1.dropna(inplace=True)

sta_lat = []
sta_lon = []
sta_vol = []

for index, row in stations1.iterrows():
    sta_lat.append(float(row['lat']))
    sta_lon.append(float(row['lon']))
    sta_vol.append(float(row['max_entries']))
    
NYC_map(comp_lat,comp_lon,comp_labels,sta_lat,sta_lon,sta_vol)
